## Optimization exercise

**Goal: Train the 2nd order polynomial predictor using gradient descent. Optimize the stepsizes and compare against scikit-learn's implementation.**

In [52]:
# IMPORT PACKAGES
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 10

#### Task I
Download the data from [here](https://drive.google.com/file/d/0Bz9_0VdXvv9bUUNlUTVrMF9VcVU/view?usp=sharing&resourcekey=0-16O9Fc5eaJH99-M7AHqHOg).

In [ ]:
data_path = ''
nb = pd.read_csv(data_path+'nba_games_2013_2015.csv', delimiter=';')
x = nb[['AST','REB','STL']]
y = nb['PTS']

In [54]:
nb.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22015,1610612750,MIN,Minnesota Timberwolves,21501226,2016-04-13,MIN vs. NOP,W,240,144,...,0.826,5,38,43,41,14,8,13,20,35.0
1,22015,1610612749,MIL,Milwaukee Bucks,21501225,2016-04-13,MIL vs. IND,L,240,92,...,0.846,7,36,43,23,8,3,15,15,-5.0
2,22015,1610612738,BOS,Boston Celtics,21501217,2016-04-13,BOS vs. MIA,W,240,98,...,0.864,10,29,39,20,7,3,7,20,10.0
3,22015,1610612747,LAL,Los Angeles Lakers,21501228,2016-04-13,LAL vs. UTA,W,239,101,...,0.867,8,39,47,19,6,3,13,17,5.0
4,22015,1610612739,CLE,Cleveland Cavaliers,21501220,2016-04-13,CLE vs. DET,L,265,110,...,0.733,8,35,43,21,4,7,10,23,-2.0


In [55]:
nb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7380 entries, 0 to 7379
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SEASON_ID          7380 non-null   int64  
 1   TEAM_ID            7380 non-null   int64  
 2   TEAM_ABBREVIATION  7380 non-null   object 
 3   TEAM_NAME          7380 non-null   object 
 4   GAME_ID            7380 non-null   int64  
 5   GAME_DATE          7380 non-null   object 
 6   MATCHUP            7380 non-null   object 
 7   WL                 7380 non-null   object 
 8   MIN                7380 non-null   int64  
 9   PTS                7380 non-null   int64  
 10  FGM                7380 non-null   int64  
 11  FGA                7380 non-null   int64  
 12  FG_PCT             7380 non-null   float64
 13  FG3M               7380 non-null   int64  
 14  FG3A               7380 non-null   int64  
 15  FG3_PCT            7380 non-null   float64
 16  FTM                7380 


#### TASK II
Create a function `psi(x)`, which transforms features AST (assists), REB (rebounds) and STL (steals) into 2nd order polynomial features (add each feature squared and each pair of features multiplied with every other)

**Input:** DataFrame x from above. It contains columns AST, REB, STL

**Output:** DataFrame with columns: AST, REB, STL, 1, AST^2, REB^2, STL^2, ASTSTL, REBSTL, ASTREB. The number of rows should be the same as Input.

In [57]:
from sklearn.preprocessing import PolynomialFeatures

In [58]:
def psi(x):
    poly = PolynomialFeatures(2)
    Xpoly = poly.fit_transform(x)
    return Xpoly

#### Task III
Create a transformed data matrix **X**, where each **x** is mapped to psi(x).

HINT: We need to apply our function from Task II to matrix (DataFrame) x

In [59]:
X = psi(x)

#### Task IV
Create a function `p2(x,w)`, which outputs the value of the polynomial ata given row of `x` for given parameters `w`.

Inputs:
- x: DataFrame from above
- w: vector which represents beta coeficients for each column of X from the Task III.

Ouputs:
- Series of the same length as DataFrame x. Each value is a dot product of particular row in DataFrame and coeficients `w`.


HINT:
- length of w needs to be the same as number of columns of the dataframe that is returned from function `psi(x)`

Example Input:

`p2(x, [0.06, 0.05,0.03,0.01,0.02,0.02,0.04, 0.03,0.02,0.01])`

Example Output:

```
0       130.37
1        76.19
2        61.21
3        74.51
4        64.97
         ...  
7375     63.01
7376     79.59
7377     97.25
7378     78.85
7379     61.53
Length: 7380, dtype: float64

```

Our columns in the DataFrame **X** that is the output of `psi(x)` were in this order: `AST, REB, STL, 1, AST^2, REB^2, STL^2, ASTSTL, REBSTL, ASTREB`. If your columns are in different order the result can be different even for the **w**.

In [60]:
def p2(x,w):
    return x@w

#### Task V
Create a function `Loss(x,y,w)`, which computes the squared loss of predicting **y** from **x** by `p2(x,w)` using parameters **w**. We have specified **y** as the variable PTS above. We will predict scored points (PTS) based on assists, rebounds and steals.


HINTS: 
- Output of `p2(x,w)` represents our predictions. `y_pred = p2(x,w)`
- Loss can be computed as:

```
np.mean((y_pred - y)**2)
```

In [61]:
def Loss(x,y,w):
    y_pred = p2(x,w)
    return np.mean((y_pred - y)**2)

#### Task VI
Code up the gradient descent. It should input **x**, target variable **y**, stepsize **alpha**, coeficients **w** and **maximum number of iterations**.

Steps:
1. transform input `x`
2. compute initial loss for given, x,y and w and append it to the empty list.
3. Inside the for loop, update each element of **w**, **w[i]**, using gradient descent.

HINT: `w[i] = w[i] - alpha * (1.0 / m) * errors_i.sum()` where `errors_i = (y_pred - y) * X.iloc[:, i]`. We are scaling the errors by multiplicating with values that are relevant for coeficients `w[i]` (column `i` of DataFrame `X`, output of `psi(x)`).

4. compute new loss using updated `w` and append to the list that we created in the step 2.
5. repeat steps 3 and 4 for max number of iterations times.

In [62]:
def gradient_descent(x,y,alpha,w,max_iterations):
    
    past_costs = []
    past_costs.append(Loss(x,y,w)) # compute initial loss for given, x,y and w and append it to the empty list for past_costs
    for i in range(max_iterations): 
        
        error = y - p2(x,w) # actual value of the target minus predicted value of the target gives us the error
        error = error.reshape(-1,1) # reshape error so that it is shape (7380,1)
        grad = -(1.0 / y.size) * (error*x).sum() # gradient is the derivative of the loss function
        w = w - alpha * grad # new coefficents are calculated by stepping 'downhill' in the direction opposite the gradient 
            
        past_costs.append(Loss(x,y,w)) # compute loss for new w and append it to the list
    
    return past_costs, w

#### Task VII
Choose an arbitrary **w** and **alpha** and run gradient descent for 100 iterations. How does the loss behave? Does it converge to something?


In [63]:
# import packages

import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import PolynomialFeatures

In [64]:
# import the data as a pandas data frame

data_path = ''
nb = pd.read_csv(data_path+'nba_games_2013_2015.csv', delimiter=';')
x = nb[['AST','REB','STL']] # features
y = nb['PTS'] # target

In [65]:
# create a function psi(x) transform features AST (assists), REB (rebounds) and STL (steals) into 2nd order polynomial features to attempt to find a function that will describe their relationship with PTS (target)

def psi(x):
    poly = PolynomialFeatures(2)
    Xpoly = poly.fit_transform(x)
    return Xpoly

X = psi(x)

In [66]:
# create a function p2(x,w), which outputs the value of the polynomial ata given row of `x` for given parameters `w`. This is the PTS prediction

def p2(x,w):
    return x@w

In [67]:
# create a function `Loss(x,y,w)`, which computes the squared loss of predicting **y** from **x** by `p2(x,w)` using parameters **w**

def Loss(x,y,w):
    y_pred = p2(x,w)
    return np.mean((y_pred - y)**2)

In [68]:
# define a function for gradient descent that returns the derivative of the loss function for a list of coefficients w, then adjusts the coefficients and repeats this for max_iterations

def gradient_descent(x,y,alpha,w,max_iterations):
    
    past_costs = []
    past_costs.append(Loss(x,y,w)) # compute initial loss for given, x,y and w and append it to the empty list for past_costs
    for i in range(max_iterations): 
        
        error = y - p2(x,w) # actual value of the target minus predicted value of the target gives us the error
        error = error.reshape(-1,1) # reshape error so that it is shape (7380,1)
        grad = -(1.0 / y.size) * (error*x).sum() # gradient is the derivative of the loss function
        w = w - alpha * grad # new coefficents are calculated by stepping 'downhill' in the direction opposite the gradient 
            
        past_costs.append(Loss(x,y,w)) # compute loss for new w and append it to the list
    
    return past_costs, w

In [69]:
w = [0.06,0.05,0.03,0.01,0.02,0.02,0.04, 0.03,0.02,0.01]
alpha = .0000001
max_iterations = 100

In [70]:
gradient_descent(X,y.values,alpha,w,max_iterations)

([524.5718913821138,
  501.9157970055969,
  490.18224285903807,
  484.10545439663974,
  480.95829561120325,
  479.3283872340995,
  478.48426035685196,
  478.04708843056363,
  477.82067780641944,
  477.70342011523906,
  477.6426925491503,
  477.6112418421992,
  477.5949535726138,
  477.5865179044902,
  477.5821490857943,
  477.5798864817518,
  477.5787146828262,
  477.5781078100832,
  477.5777935116842,
  477.5776307370584,
  477.57754643634655,
  477.57750277714604,
  477.5774801661176,
  477.5774684559031,
  477.5774623912022,
  477.57745925030343,
  477.5774576236372,
  477.57745678118937,
  477.5774563448871,
  477.57745611892676,
  477.5774560019023,
  477.57745594129557,
  477.5774559099074,
  477.5774558936516,
  477.5774558852327,
  477.5774558808726,
  477.5774558786145,
  477.577455877445,
  477.57745587683934,
  477.5774558765257,
  477.57745587636316,
  477.57745587627903,
  477.5774558762355,
  477.57745587621287,
  477.57745587620127,
  477.5774558761952,
  477.57745587619

#### Task VIII
Can you find which **alpha** from `[1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]` has the smallest loss after 100 iterations for a given **w**?

In [71]:
# 0.0000001 has the smallest loss after 100 iterations

alpha_list = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]

for alpha in alpha_list:
    print(f'The loss with alpha {alpha}:\n {gradient_descent(X,y.values,alpha,w,max_iterations)[0]} \n The weights after {max_iterations} iterations:\n {gradient_descent(X,y.values,alpha,w,max_iterations)[1]}', end='\n'*2)

C:\Users\User\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:180: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\Users\User\AppData\Local\Temp\ipykernel_7388\3328901806.py:5: RuntimeWarning: overflow encountered in square
  return np.mean((y_pred - y)**2)
C:\Users\User\AppData\Local\Temp\ipykernel_7388\915277839.py:11: RuntimeWarning: overflow encountered in multiply
  grad = -(1.0 / y.size) * (error*x).sum() # gradient is the derivative of the loss function
C:\Users\User\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


The loss with alpha 1:
 [524.5718913821138, 1.3897199768073922e+16, 4.1096814828085395e+30, 1.2153154716060302e+45, 3.5939322833253174e+59, 1.0627980601661458e+74, 3.1429076221987286e+88, 9.294209965090328e+102, 2.748484819122767e+117, 8.127822406984853e+131, 2.4035605588889487e+146, 7.107811995599032e+160, 2.1019229649921927e+175, 6.215808962725982e+189, 1.838139727506528e+204, 5.435748875325793e+218, 1.6074602704815697e+233, 4.753583324840064e+247, 1.4057301969539766e+262, 4.1570269238832646e+276, 1.2293164707804968e+291, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan] 
 The weights after 100 iterations:
 [nan nan nan nan nan nan nan nan 